# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

In [ ]:
from langchain.agents import create_agent
agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    system_prompt="You are a helpful banker. Answer customer queries with warmth and professionalism.",
)

## No Steaming (invoke)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "What are the requirements and benefits of opening a business banking account versus a standard personal account?"}]})
print(result["messages"][1].content)

## values
You have seen this streaming mode in our examples so far. 

In [ ]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "What are the requirements and benefits of opening a business banking account versus a standard personal account?"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [ ]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Are your ATM networks part of any larger alliances that provide surcharge-free access to a wider range of ATMs? And if so, which ones? Explain me in detail"}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [7]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer
from langchain_core.tools import tool


@tool
def compare_interest_rates(product: str) -> str:
    """GetCompare interest rates for a given bank product with competitors."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up current interest rates for product: {product}")
    writer(f"Looking up interest rates for competitor product: {product}")
    return f"We offer the most competitive interest rates for {product}."


agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[compare_interest_rates],
    system_prompt="You are a helpful banker. Answer customer queries with warmth and professionalism.",
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Compare interest rates for savings accounts."}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='Compare interest rates for savings accounts.', additional_kwargs={}, response_metadata={}, id='312b52a4-fcc9-4bbb-b2be-3e2ea3a514cf')]})
('values', {'messages': [HumanMessage(content='Compare interest rates for savings accounts.', additional_kwargs={}, response_metadata={}, id='312b52a4-fcc9-4bbb-b2be-3e2ea3a514cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'f9c01kd4p', 'function': {'arguments': '{"product":"savings accounts"}', 'name': 'compare_interest_rates'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 245, 'total_tokens': 263, 'completion_time': 0.024587559, 'completion_tokens_details': None, 'prompt_time': 0.013464138, 'prompt_tokens_details': None, 'queue_time': 0.054967411, 'total_time': 0.038051697}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, '

## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "insert your query here"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])